In [1]:
import tensorflow as tf
from tensorflow import keras

import os
import tempfile

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import imblearn
import collections

import sklearn
from sklearn.metrics import *
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

from sklearn.neural_network import MLPClassifier

from sklearn.metrics import classification_report,confusion_matrix

from keras.utils import to_categorical

from keras.utils.vis_utils import plot_model

from collections import Counter
from imblearn.under_sampling import OneSidedSelection
from imblearn.over_sampling import ADASYN
import pickle

Load the Data

In [2]:
csv_file = 'TMomoUBharat_alldata_final_doaction.csv'
dataframe = pd.read_csv(csv_file)

In [3]:
dataframe.head()

print(dataframe["Time Since Last Charge"].mean())
print(dataframe["Time Since Last Action Formatted"].mean())

27.14862307940089
2.2738761147522863


Format and One-hot encode the data

In [4]:
dataframe.drop("tsla1", axis=1, inplace=True)
dataframe.drop("tsla2", axis=1, inplace=True)
dataframe.drop("tsla3", axis=1, inplace=True)
dataframe.drop("tsla4", axis=1, inplace=True)
dataframe.drop("tsla5", axis=1, inplace=True)
dataframe.drop("tsla6", axis=1, inplace=True)
dataframe.drop("st1", axis=1, inplace=True)
dataframe.drop("st2", axis=1, inplace=True)
dataframe.drop("st3", axis=1, inplace=True)
dataframe.drop("st4", axis=1, inplace=True)
dataframe.drop("st5", axis=1, inplace=True)
dataframe.drop("st6", axis=1, inplace=True)
dataframe.drop("st7", axis=1, inplace=True)
dataframe.drop("st8", axis=1, inplace=True)
dataframe.drop("st9", axis=1, inplace=True)
dataframe.drop("st10", axis=1, inplace=True)
dataframe.drop("Session Time", axis=1, inplace=True)
dataframe.drop("Time Since Last Action", axis=1, inplace=True)
dataframe.drop("Action", axis=1, inplace=True)

dataframe.drop("Session Time Formatted", axis=1, inplace=True)
#dataframe.drop("Time Since Last Charge", axis=1, inplace=True)  

In [5]:
dataframe.head(100)

,Do Action,Last Action,Time Since Last Charge,Time Since Last Action Formatted,Camera Mood,Gaze,Noise,Input Mood,Activity
0,1,17,0.00,0.02,Sad,1,Low,Neutral,Studying
1,0,2,0.00,0.00,Sad,1,Low,Neutral,Studying
2,0,2,0.00,0.00,Sad,1,Low,Neutral,Studying
3,0,2,0.00,0.00,Sad,1,Low,Neutral,Studying
4,0,2,0.00,0.00,Sad,1,Low,Neutral,Studying
...,...,...,...,...,...,...,...,...,...
95,0,10,0.30,0.07,Sad,1,Low,Neutral,Studying
96,0,10,0.32,0.08,Sad,1,Low,Neutral,Studying
97,0,10,0.32,0.08,Sad,1,Low,Neutral,Studying
98,0,10,0.32,0.08,Neutral,0,Low,Neutral,Studying


In [6]:
dataframe = pd.get_dummies(dataframe, columns=["Activity"], prefix=["Activity"])
dataframe = pd.get_dummies(dataframe, columns=["Camera Mood"], prefix=["Camera_Mood"])
dataframe = pd.get_dummies(dataframe, columns=["Input Mood"], prefix=["Input Mood"])
#both = pd.concat([dataframe, dum_df])
#dataframe.drop("Input Mood", axis=1, inplace=True) 
#both.head(5)

dataframe = pd.get_dummies(dataframe, columns=["Noise"], prefix=["Noise"])
dataframe = pd.get_dummies(dataframe, columns=["Last Action"], prefix=["Last Action"])
dataframe.head(100)

,Do Action,Time Since Last Charge,Time Since Last Action Formatted,Gaze,Activity_Eating,Activity_Relaxing,Activity_Studying,Camera_Mood_Happy,Camera_Mood_Neutral,Camera_Mood_Sad,...,Last Action_7,Last Action_8,Last Action_9,Last Action_10,Last Action_11,Last Action_12,Last Action_13,Last Action_14,Last Action_16,Last Action_17
0,1,0.00,0.02,1,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,1
1,0,0.00,0.00,1,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,0,0.00,0.00,1,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,0,0.00,0.00,1,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,0,0.00,0.00,1,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0,0.30,0.07,1,0,0,1,0,0,1,...,0,0,0,1,0,0,0,0,0,0
96,0,0.32,0.08,1,0,0,1,0,0,1,...,0,0,0,1,0,0,0,0,0,0
97,0,0.32,0.08,1,0,0,1,0,0,1,...,0,0,0,1,0,0,0,0,0,0
98,0,0.32,0.08,0,0,0,1,0,1,0,...,0,0,0,1,0,0,0,0,0,0


In [7]:
X = dataframe.drop('Do Action', axis=1)
y = dataframe['Do Action']

In [8]:
#X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.3)
X_train = X[18569:74456]
X_test = X[0:18569]
y_train = y[18569:74456]
y_test = y[0:18569]

In [9]:
X_train

,Time Since Last Charge,Time Since Last Action Formatted,Gaze,Activity_Eating,Activity_Relaxing,Activity_Studying,Camera_Mood_Happy,Camera_Mood_Neutral,Camera_Mood_Sad,Input Mood_Negative,...,Last Action_7,Last Action_8,Last Action_9,Last Action_10,Last Action_11,Last Action_12,Last Action_13,Last Action_14,Last Action_16,Last Action_17
18569,0.00,0.02,1,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
18570,0.00,0.00,1,0,0,1,0,0,1,0,...,1,0,0,0,0,0,0,0,0,0
18571,0.00,0.00,1,0,0,1,0,0,1,0,...,1,0,0,0,0,0,0,0,0,0
18572,0.00,0.00,1,0,0,1,0,0,1,0,...,1,0,0,0,0,0,0,0,0,0
18573,0.00,0.00,1,0,0,1,0,0,1,0,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74451,62.83,13.38,1,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
74452,62.83,13.38,1,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
74453,62.85,13.38,1,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
74454,62.85,13.38,1,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1


In [10]:
counter = Counter(y_train)
print(counter)

Counter({0: 55727, 1: 160})


In [11]:
counter = Counter(y_test)
print(counter)

Counter({0: 18493, 1: 76})


In [12]:
X_train_scaled = X_train.copy()

col_names = ['Time Since Last Charge', 'Time Since Last Action Formatted']
features = X_train_scaled[col_names]
scaler = StandardScaler().fit(features.values)
features = scaler.transform(features.values)
X_train_scaled[col_names] = features

filename = 'TMomoUBharat_scaler1.sav'
#pickle.dump(scaler, open(filename, 'wb'))

In [13]:
pd.set_option('display.max_columns', None)
X_train_scaled.head()

,Time Since Last Charge,Time Since Last Action Formatted,Gaze,Activity_Eating,Activity_Relaxing,Activity_Studying,Camera_Mood_Happy,Camera_Mood_Neutral,Camera_Mood_Sad,Input Mood_Negative,Input Mood_Neutral,Input Mood_Positive,Noise_High,Noise_Low,Noise_None,Last Action_1,Last Action_2,Last Action_3,Last Action_4,Last Action_5,Last Action_6,Last Action_7,Last Action_8,Last Action_9,Last Action_10,Last Action_11,Last Action_12,Last Action_13,Last Action_14,Last Action_16,Last Action_17
18569,-1.719807,-0.601431,1,0,0,1,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
18570,-1.719807,-0.606126,1,0,0,1,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
18571,-1.719807,-0.606126,1,0,0,1,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
18572,-1.719807,-0.606126,1,0,0,1,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
18573,-1.719807,-0.606126,1,0,0,1,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0


In [14]:
scaler2 = pickle.load(open(filename, 'rb'))

X_test_scaled = X_test.copy()

features = X_test_scaled[col_names]
features = scaler2.transform(features.values)
X_test_scaled[col_names] = features

X_test_scaled.head()

,Time Since Last Charge,Time Since Last Action Formatted,Gaze,Activity_Eating,Activity_Relaxing,Activity_Studying,Camera_Mood_Happy,Camera_Mood_Neutral,Camera_Mood_Sad,Input Mood_Negative,Input Mood_Neutral,Input Mood_Positive,Noise_High,Noise_Low,Noise_None,Last Action_1,Last Action_2,Last Action_3,Last Action_4,Last Action_5,Last Action_6,Last Action_7,Last Action_8,Last Action_9,Last Action_10,Last Action_11,Last Action_12,Last Action_13,Last Action_14,Last Action_16,Last Action_17
0,-1.719807,-0.601431,1,0,0,1,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,-1.719807,-0.606126,1,0,0,1,0,0,1,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,-1.719807,-0.606126,1,0,0,1,0,0,1,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,-1.719807,-0.606126,1,0,0,1,0,0,1,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,-1.719807,-0.606126,1,0,0,1,0,0,1,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


OSS Undersampling

In [15]:
# define the undersampling method
undersample = OneSidedSelection(n_neighbors=1, n_seeds_S=200)
# transform the dataset
X_train_scaled, y_train = undersample.fit_resample(X_train_scaled, y_train)
# summarize the new class distribution
counter = Counter(y_train)
print(counter)

Counter({0: 19189, 1: 160})


SMOTE ADASYN Oversampling

In [16]:
from imblearn.over_sampling import SMOTE

# define the undersampling method
oversample = SMOTE()        ####################33
# transform the dataset
X_train_scaled, y_train = oversample.fit_resample(X_train_scaled, y_train)
# summarize the new class distribution
counter = Counter(y_train)
print(counter)

Counter({0: 19189, 1: 19189})


Load and Train the Model

In [17]:
mlp = MLPClassifier(max_iter=10000, activation='relu', hidden_layer_sizes=(15)) #doesn't converge at 500 so put to higher, do we want it to converge?
mlp

MLPClassifier(hidden_layer_sizes=15, max_iter=10000)

In [18]:
mlp.fit(X_train_scaled,y_train)

MLPClassifier(hidden_layer_sizes=15, max_iter=10000)

In [59]:
#filename = 'TMomoUBharat_model1.sav'
#pickle.dump(mlp, open(filename, 'wb'))

In [19]:
pred = mlp.predict(X_test_scaled)
pred

array([1, 0, 0, ..., 0, 0, 0])

In [20]:
print(classification_report(y_test,pred))

              precision    recall  f1-score   support

           0       1.00      0.92      0.96     18493
           1       0.00      0.08      0.01        76

    accuracy                           0.92     18569
   macro avg       0.50      0.50      0.48     18569
weighted avg       0.99      0.92      0.95     18569



In [21]:
y_test.value_counts()

0    18493
1       76
Name: Do Action, dtype: int64

In [22]:
from collections import Counter
Counter(pred)

Counter({1: 1411, 0: 17158})

In [113]:
prediction = pd.DataFrame(pred, columns=['predictions']).to_csv('predictmlp.csv')

In [53]:
y_test.to_csv('actual.csv')